In [2]:
%matplotlib widget

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pydae.ssa as ssa
import scipy.signal as sctrl

In [8]:
from vsc_lcl import vsc_lcl_class 

## Instantiate system

In [45]:
syst = vsc_lcl_class()
syst.Dt = 5e-6
syst.decimation = 1
syst.N_store = 100_000
syst.update()

## CTRL1 in state feedback

In [82]:
syst = vsc_lcl_class()
syst.Dt = 5e-6
syst.decimation =1
syst.N_store =100_000
syst.update()

Δt = 50e-6 
#x_d_ctrl_list = ['i']   # states to consider in the reduction
z_ctrl_list = [ 'i_sd_g01', 'i_sq_g01']     # outputs to consider in the controller
u_ctrl_list = ['eta_d_g01','eta_q_g01']     # intputs to consider in the controller
z_ctrl_idxs = [syst.outputs_list.index(item) for item in z_ctrl_list]
u_ctrl_idxs = [syst.inputs_run_list.index(item) for item in u_ctrl_list]

syst.Δt = Δt

## Calculate equilibirum point
syst.initialize([{'G_d_g01':0.0,'eta_d_g01':0.0,'eta_q_g01':-0.8693333,'v_1_Q':-326,'v_1_D':0.0, 'C_m_g01':4e-6}],xy0=1000)
ssa.eval_ss(syst)

# linear continous plant
A_p = syst.A
B_p = syst.B
C_p = syst.C
D_p = syst.D

# plant discretization
A_d,B_d,C_d,D_d,Dt = sctrl.cont2discrete((A_p,B_p,C_p,D_p),Δt,method='zoh')

N_z_d,N_x_d = C_d.shape  # discreticed plant dimensions
N_x_d,N_u_d = B_d.shape

# convenient matrices
O_ux = np.zeros((N_u_d,N_x_d))
O_xu = np.zeros((N_x_d,N_u_d))
O_uu = np.zeros((N_u_d,N_u_d))
I_uu = np.eye(N_u_d)

syst.A_d = A_d
syst.B_d = B_d


# Controller ##################################################################################
B_c = B_d[:,u_ctrl_idxs]
C_c = C_d[z_ctrl_idxs,:]
D_c = D_d[z_ctrl_idxs,:][:,u_ctrl_idxs]

N_x_c,N_u_d = B_c.shape
N_z_c,N_x_c = C_c.shape


O_ux = np.zeros((N_u_d,N_x_d))
O_xu = np.zeros((N_x_d,N_u_d))
O_uu = np.zeros((N_u_d,N_u_d))
I_uu = np.eye(N_u_d)


# discretized plant:
# Δx_d = A_d*Δx_d + B_d*Δu_d
# Δz_c = C_c*Δx_d + D_c*Δu_d

# dinamic extension:
# Δx_d = A_d*Δx_d + B_d*Δu_d
# Δx_i = Δx_i + Δt*(Δz_c-Δz_c_ref) = Δx_i + Δt*C_c*Δx_d - Dt*Δz_c_ref
# Δz_c = z_c - z_c_0
# Δz_c_ref = z_c_ref - z_c_0
# (Δz_c-Δz_c_ref) = z_c - z_c_ref
omega_b = 2*np.pi*50

W =  np.block([
                [   np.cos(omega_b*Δt), -np.sin(omega_b*Δt)],   
                [   np.sin(omega_b*Δt),  np.cos(omega_b*Δt)],    
               ])

A_e = np.block([
                [    A_d,  B_c@W, O_xu],    # Δx_d
                [   O_ux,   O_uu, O_uu],    # Δx_r
                [ Δt*C_c, Δt*D_c, I_uu],    # Δx_i    
               ])

B_e = np.block([
                [   O_xu],
                [   I_uu],
                [   O_uu],    
               ])

A_ctrl = A_e[N_x_d:,N_x_d:]
B_ctrl = B_e[N_x_d:]

# weighting matrices
Q_c = np.eye(A_e.shape[0])
Q_c[-1,-1] = 1e6
Q_c[-2,-2] = 1e6

R_c = np.eye(B_c.shape[1])*100000

K_c,S_c,E_c = ssa.dlqr(A_e,B_e,Q_c,R_c)

E_cont = np.log(E_c)/Δt

syst.A_ctrl = A_ctrl
syst.B_ctrl = B_ctrl
syst.K_c = K_c
syst.N_x_d = N_x_d  # number of plant states
syst.N_u_d = N_u_d  # number of plant inputs
syst.N_z_c = N_z_c  # number of plant outputs considered for the controller

In [83]:
syst = vsc_lcl_class()
syst.Dt = 5e-6
syst.decimation =1
syst.N_store =100_000
syst.update()
times = np.arange(0.0,0.1,Δt)

syst.initialize([{'G_d_g01':0.0,'eta_d_g01':0.0,'eta_q_g01':-0.8693333,'v_1_Q':-326,'v_1_D':0.0, 'C_m_g01':4e-6}],xy0=1000)
ssa.eval_A(syst)
i_sd = syst.get_value('i_sd_g01')
i_sq = syst.get_value('i_sq_g01')  
v_sd = syst.get_value('v_sd_g01')
v_sq = syst.get_value('v_sq_g01')
i_td = syst.get_value('i_td_g01')
i_tq = syst.get_value('i_tq_g01') 
v_md = syst.get_value('v_md_g01')
v_mq = syst.get_value('v_mq_g01') 
v_dc = syst.get_value('v_dc_g01')
eta_d = syst.get_value('eta_d_g01')
eta_q = syst.get_value('eta_q_g01')
i_sd_ref_0 = i_sd
i_sq_ref_0 = i_sq
v_sq_0 = v_sq
v_sd_0 = v_sd
x_d_0 = np.array([i_td,i_tq,v_md,v_mq,i_sd,i_sq]).reshape(6,1)
u_d_0 = np.array([eta_d,eta_q]).reshape(2,1)
x_r_0 = u_d_0
syst.Δx_e = np.zeros((10,1))
it = 0
for t in times:
    
    Δx_e = syst.Δx_e
    # measurements
    i_sd = syst.get_value('i_sd_g01')
    i_sq = syst.get_value('i_sq_g01')  
    v_sd = syst.get_value('v_sd_g01')
    v_sq = syst.get_value('v_sq_g01')
    i_td = syst.get_value('i_td_g01')
    i_tq = syst.get_value('i_tq_g01') 
    v_md = syst.get_value('v_md_g01')
    v_mq = syst.get_value('v_mq_g01') 
    v_dc = syst.get_value('v_dc_g01')

    x_d = np.array([i_td,i_tq,v_md,v_mq,i_sd,i_sq]).reshape(6,1)
    
    Δx_d = x_d - x_d_0     
    Δx_r = syst.Δx_e[N_x_c:-N_u_d,:]   
    Δx_i = syst.Δx_e[(N_x_c+N_u_d):,:] 
    
    i_sd_ref = i_sd_ref_0
    i_sq_ref = i_sq_ref_0
    v_sq = v_sq_0
    v_sd = v_sd_0
    if t>20e-3: i_sd_ref = 20
    if t>30e-3: i_sq_ref = 30
    if t>45e-3: v_sd =  163 
    if t>45e-3: v_sq = -163
    epsilon_d = i_sd - i_sd_ref
    epsilon_q = i_sq - i_sq_ref  
    
    epsilon = np.block([[epsilon_d],[epsilon_q]])
    
    Δu_r = -K_c @ Δx_e + np.block([[ (v_sd-v_sd_0)*2/v_dc],[(v_sq-v_sq_0)*2/v_dc]])
    
   
    Δx_r = W@Δu_r
    Δx_i += Δt*epsilon
    
    Δx_e = np.block([[Δx_d],[Δx_r],[Δx_i]])
    
    syst.Δx_e  =  Δx_e
    
    x_r = Δx_r + x_r_0 
    
    eta_dq = x_r
    eta_d = eta_dq[0,0]  
    eta_q = eta_dq[1,0]   

    
    events=[{'t_end':t,'eta_d_g01':eta_d,'eta_q_g01':eta_q,'v_1_Q':v_sq,'v_1_D':v_sd}]
    syst.run(events)

#    eta_d_prev = eta_d
#    eta_q_prev = eta_q
    it += 1
    
syst.post();



In [84]:
plt.close('all')
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(7, 7),sharex=True)

lines = axes[0].plot(syst.T,syst.get_values('i_sd_g01'),label='i_sd_g01')
lines = axes[0].plot(syst.T,syst.get_values('i_sq_g01'),label='i_sq_g01')

axes[1].plot(syst.T,syst.get_values('eta_D_g01'),label='eta_D_g01')
axes[1].plot(syst.T,syst.get_values('eta_Q_g01'),label='eta_Q_g01')
for ax in axes:
    ax.grid()
    ax.legend()
ax.set_xlabel('Time (s)')

datacursor(lines, display='multiple')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [85]:
import sympy as sym

x_d_1,x_d_2,x_d_3,x_d_4,x_d_5,x_d_6 = sym.symbols('Dx_d_1,Dx_d_2,Dx_d_3,Dx_d_4,Dx_d_5,Dx_d_6')
x_r_1,x_r_2 = sym.symbols('Dx_r_1,Dx_r_2')
x_i_1,x_i_2 = sym.symbols('Dx_i_1,Dx_i_2')

x_e = sym.Matrix([x_d_1,x_d_2,x_d_3,x_d_4,x_d_5,x_d_6,x_r_1,x_r_2,x_i_1,x_i_2])
u_r = -K_c * x_e

In [86]:
u_r_d = str(sym.N(u_r[0],8))
u_r_q = str(sym.N(u_r[1],8))

print(f'Du_r_1 = {u_r_d};')
print(f'Du_r_2 = {u_r_q};')

Du_r_1 = -0.013355528*Dx_d_1 - 0.000487326*Dx_d_2 + 0.0021855202*Dx_d_3 + 8.7205357e-5*Dx_d_4 + 0.0071938875*Dx_d_5 + 0.00034010605*Dx_d_6 - 2.2549315*Dx_i_1 + 0.65433868*Dx_i_2 - 0.36460717*Dx_r_1 - 0.0049366387*Dx_r_2;
Du_r_2 = 0.000487326*Dx_d_1 - 0.013355528*Dx_d_2 - 8.7205357e-5*Dx_d_3 + 0.0021855202*Dx_d_4 - 0.00034010605*Dx_d_5 + 0.0071938875*Dx_d_6 - 0.65433868*Dx_i_1 - 2.2549315*Dx_i_2 + 0.0049366387*Dx_r_1 - 0.36460717*Dx_r_2;


In [66]:
Du_r_1,Du_r_2 = sym.symbols('Du_r_1,Du_r_2')
Du_r = sym.Matrix([Du_r_1,Du_r_2 ])

Dx_r = W@Du_r

Dx_r_1 = str(sym.N(Dx_r[0],8))
Dx_r_1 = str(sym.N(Dx_r[1],8))

print(f'Dx_r_1 = {u_r_d};')
print(f'Dx_r_2 = {u_r_q};')

Matrix([
[0.999876632481661*Du_r_1 - 0.0157073173118207*Du_r_2],
[0.0157073173118207*Du_r_1 + 0.999876632481661*Du_r_2]])

In [87]:
print(u_r[0])

0.023914820787791*Dx_d_1 + 0.00124312201525765*Dx_d_2 + 0.00210868343440974*Dx_d_3 + 9.76673467759675e-5*Dx_d_4 - 0.0363072666129747*Dx_d_5 - 0.0013695946212219*Dx_d_6 - 10.022203616505*Dx_i_1 + 1.45742192739155*Dx_i_2 - 0.327372378893965*Dx_r_1 - 0.000502327486946109*Dx_r_2


In [88]:
print(u_r[1])

-0.00124312201525856*Dx_d_1 + 0.0239148207877911*Dx_d_2 - 9.76673467759516e-5*Dx_d_3 + 0.00210868343440976*Dx_d_4 + 0.00136959462122251*Dx_d_5 - 0.0363072666129741*Dx_d_6 - 1.45742192739181*Dx_i_1 - 10.0222036165041*Dx_i_2 + 0.000502327486927267*Dx_r_1 - 0.327372378893964*Dx_r_2


In [63]:
syst.get_value('C_m_g01')

4e-06

In [32]:
from mpldatacursor import datacursor

data = np.outer(range(10), range(1, 5))

fig, ax = plt.subplots()
lines = ax.plot(data)
ax.set_title('Click somewhere on a line')

#datacursor(lines)
datacursor(display='multiple', draggable=True)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [30]:
import matplotlib.pyplot as plt
import numpy as np
from mpldatacursor import datacursor

data = np.outer(range(10), range(1, 5))

plt.plot(data)
plt.title('Click somewhere on a line')

datacursor()

plt.show()

In [ ]:
Ts_ctr = 1/200;
Ts_med = 1/20000;

wN_ctr = 2*pi*1/Ts_ctr/2;
wN_med = 2*pi*1/Ts_med/2;

[nA, Wn] = buttord(wN_ctr, wN_med, -20*log10(0.7), -20*log10(0.1), 's');
[NUM_aaA,DEN_aaA] = butter(nA,Wn,'low','s');

In [88]:
from scipy import signal
import matplotlib.pyplot as plt






In [97]:
plt.close('all')
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(7, 7),sharex=True)

N, Wn = signal.buttord([20, 50], [14, 60], 3, 40, True)
b, a = signal.butter(N, Wn, 'band', True)
w, h = signal.freqs(b, a, np.logspace(1, 2, 500))
axes.plot(w/2/np.pi, 20 * np.log10(abs(h)))
plt.title('Butterworth bandpass filter fit to constraints')
axes.set_xlabel('Frequency [radians / second]')
axes.set_ylabel('Amplitude [dB]')
axes.grid(which='both', axis='both')
axes.fill([1,  14,  14,   1], [-40, -40, 99, 99], '0.9', lw=0) # stop
axes.fill([20, 20,  50,  50], [-99, -3, -3, -99], '0.9', lw=0) # pass
axes.fill([60, 60, 1e9, 1e9], [99, -40, -40, 99], '0.9', lw=0) # stop
axes.set_xlim([0, 20e3])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0, 20000.0)

In [98]:
b

array([1.59811317e+19, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00])

In [99]:
a

array([1.00000000e+00, 2.48932358e+02, 4.39836594e+04, 5.53273916e+06,
       5.72496657e+08, 4.90725288e+10, 3.64084365e+12, 2.34845893e+14,
       1.34274060e+16, 6.81861593e+17, 3.10677599e+19, 1.27024773e+21,
       4.68749241e+22, 1.55890260e+24, 4.68749241e+25, 1.27024773e+27,
       3.10677599e+28, 6.81861593e+29, 1.34274060e+31, 2.34845893e+32,
       3.64084365e+33, 4.90725288e+34, 5.72496657e+35, 5.53273916e+36,
       4.39836594e+37, 2.48932358e+38, 1.00000000e+39])